In [ ]:
# Install all the requirements with:
! sudo apt-get install libboost-dev libboost-serialization-dev \
gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig

#! sudo apt-get install libboost-dev libboost-serialization-dev \
#gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

In [ ]:
!git clone https://github.com/cyang-kth/fmm.git

In [ ]:
# !!!! This could take ~ 6 mintues !!!!
# it may not work and requires for password, etc.
# in this case, go to your folder using console to compile and install FMM

import os
# change working directory
os.chdir("fmm")

if not os.path.exists('build'):
  os.mkdir('build')
# ! mkdir build
os.chdir("build")
# ! cd build
! cmake ..
! make -j4
! sudo make install

In [ ]:
import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import numpy as np

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)

    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')

    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

In [ ]:
place ="Porto, Portugal"

start_time = time.time()
G = ox.graph_from_place(place, network_type='drive', which_result=2)

save_graph_shapefile_directional(G, filepath='/content/data/porto')
print("--- %s seconds ---" % (time.time() - start_time))

ox.plot_graph(G)
ox.save_graphml(G)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import os
import json
import csv
from fmm import (
    Network,
    NetworkGraph,
    UBODTGenAlgorithm,
    UBODT,
    FastMapMatch,
    FastMapMatchConfig,
    STMATCH,
    STMATCHConfig
)



# Set up paths to save to Google Drive
google_drive_path = '/content/drive/My Drive'  # Adjust if your Google Drive path is different

data_dir = '/content/data'

save_dir = os.path.join(google_drive_path, 'data')

# Ensure the data directory exists
os.makedirs(data_dir, exist_ok=True)

# Paths to your data and output files in Google Drive
network_file_path = os.path.join(data_dir, "porto", "edges.shp")
ubodt_file_path = os.path.join(data_dir, "ubodt.txt")
train_csv_path = os.path.join(data_dir, "train-1500-cleaned.csv")
output_csv_path = os.path.join(save_dir, "matched-results-1500_cleaned.csv")

# Read network data
network = Network(network_file_path, "fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(), network.get_edge_count()))
graph = NetworkGraph(network)

# Precompute an UBODT table
if os.path.isfile(ubodt_file_path):
    ubodt = UBODT.read_ubodt_csv(ubodt_file_path)
    print("Read the ubodt file")
else:
    print("Generate and read the ubodt file")
    ubodt_gen = UBODTGenAlgorithm(network, graph)
    status = ubodt_gen.generate_ubodt(ubodt_file_path, 0.03, binary=False, use_omp=True)
    print(f"UBODT Generation Status: {status}")
    ubodt = UBODT.read_ubodt_csv(ubodt_file_path)

# Create FMM model
fmm_model = FastMapMatch(network, graph, ubodt)

# Define map matching configurations for FMM
k = 16
radius = 0.005
gps_error = 0.0005
fmm_config = FastMapMatchConfig(k, radius, gps_error)

# Create STMATCH model
stmatch_model = STMATCH(network, graph)

# Define STMATCH map matching configurations
k_stmatch = 8
radius_stmatch = 0.05
gps_error_stmatch = 0.005
vmax = 0.003
factor = 1.5
stmatch_config = STMATCHConfig(k_stmatch, radius_stmatch, gps_error_stmatch, vmax, factor)

# Read trajectory data from CSV
train1500 = []
with open(train_csv_path, "r", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        polyline_str = row.get("POLYLINE")
        if polyline_str:
            try:
                row["POLYLINE"] = json.loads(polyline_str)
            except json.JSONDecodeError:
                print(f"Invalid POLYLINE format in row {row}")
                row["POLYLINE"] = []
        else:
            row["POLYLINE"] = []
        train1500.append(row)


for i, trajectory in enumerate(train1500):
    polyline = trajectory["POLYLINE"]
    if not polyline:
        print(f"Empty POLYLINE for trajectory index {i}")
        continue

    # Convert polyline to WKT format
    wkt = "LINESTRING(" + ",".join(["{} {}".format(point[0], point[1]) for point in polyline]) + ")"

    # Try FMM
    result = fmm_model.match_wkt(wkt, fmm_config)
    train1500[i]["MATCHING_ALGORITHM"] = "fmm"

    if not list(result.cpath):
        # Try STMATCH if FMM fails
        result = stmatch_model.match_wkt(wkt, stmatch_config)
        train1500[i]["MATCHING_ALGORITHM"] = "stmatch"
        if not list(result.cpath):
            print(f"No match found for trajectory index {i}, WKT: {wkt}")
            continue  # Skip to next trajectory if no match found

    # Adding matched results according to your output field specifications
    train1500[i]["MATCHED_RESULTS"] = {
        "id": i,
        "ogeom": wkt,
        "opath": list(result.opath),
        "error": [c.error for c in result.candidates],
        "offset": [c.offset for c in result.candidates],
        "length": [c.length for c in result.candidates],
        "spdist": [c.spdist for c in result.candidates],
        "duration": [0] * len(result.candidates),  # Placeholder, replace with actual duration if available
        "speed": [0] * len(result.candidates),  # Placeholder, replace with actual speed if available
        "pgeom": result.pgeom.export_wkt(),
        "cpath": list(result.cpath),
        "tpath": list(result.indices),
        "mgeom": result.mgeom.export_wkt(),
        "ep": [c.ep for c in result.candidates],
        "tp": [c.tp for c in result.candidates],
        "matching_algorithm": train1500[i]["MATCHING_ALGORITHM"],
        "eid": [c.edge_id for c in result.candidates],
        "source": [c.source for c in result.candidates],
        "target": [c.target for c in result.candidates]
    }
    print(f"Processed trajectory index {i}")

# Save the results to a new CSV file
output_dir = os.path.dirname(output_csv_path)

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

# Define CSV headers based on the modified field names
headers = ["id", "ogeom", "opath", "error", "offset", "length", "spdist", "duration", "speed",
           "pgeom", "cpath", "tpath", "mgeom", "ep", "tp", "MATCHING_ALGORITHM", "eid", "source", "target"]

with open(output_csv_path, "w", newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()

    for trajectory in train1500:  
        if "MATCHED_RESULTS" in trajectory:
            matched_results = trajectory["MATCHED_RESULTS"]
            flattened_row = {
                "id": matched_results["id"],
                "ogeom": matched_results["ogeom"],
                "opath": json.dumps(matched_results["opath"]),
                "error": json.dumps(matched_results["error"]),
                "offset": json.dumps(matched_results["offset"]),
                "length": json.dumps(matched_results["length"]),
                "spdist": json.dumps(matched_results["spdist"]),
                "duration": json.dumps(matched_results["duration"]),
                "speed": json.dumps(matched_results["speed"]),
                "pgeom": matched_results["pgeom"],
                "cpath": json.dumps(matched_results["cpath"]),
                "tpath": json.dumps(matched_results["tpath"]),
                "mgeom": matched_results["mgeom"],
                "ep": json.dumps(matched_results["ep"]),
                "tp": json.dumps(matched_results["tp"]),
                "MATCHING_ALGORITHM": matched_results["matching_algorithm"],
                "eid": json.dumps(matched_results["eid"]),
                "source": json.dumps(matched_results["source"]),
                "target": json.dumps(matched_results["target"])
            }
            writer.writerow(flattened_row)

print(f"Results saved to {output_csv_path}")


In [2]:
import folium
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString
import os

def parse_mgeom(value):
    """Parse LINESTRING format into a LineString object."""
    try:
        if value.startswith("LINESTRING"):
            coords = value.replace("LINESTRING(", "").replace(")", "").split(",")
            coords = [(float(coord.split()[0]), float(coord.split()[1])) for coord in coords]
            return LineString(coords)
        return None
    except Exception as e:
        print(f"Error parsing geometry: {e}")
        return None

def calculate_boundaries(current_x_max, current_y_max, current_x_min, current_y_min, is_first, longitude, latitude):
    if is_first:
        current_x_min = current_x_max = longitude
        current_y_min = current_y_max = latitude
        is_first = False
    else:
        current_x_max = max(current_x_max, longitude)
        current_x_min = min(current_x_min, longitude)
        current_y_max = max(current_y_max, latitude)
        current_y_min = min(current_y_min, latitude)
    return current_x_max, current_y_max, current_x_min, current_y_min, is_first

def visualize_routes(file_path, save_combined=True, save_individual=True, plot_lines=True, plot_points=True):
    """
    Visualize the first 15 routes from the CSV file by creating a combined map and individual maps.

    Parameters:
    - file_path (str): Path to the CSV file containing route data.
    - save_combined (bool): Whether to save the combined map.
    - save_individual (bool): Whether to save individual maps for each route.
    - plot_lines (bool): Whether to plot lines between points in the routes.
    - plot_points (bool): Whether to plot individual points in the routes.
    """
    # Check if the file exists
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return

    # Load and parse geometries
    df = pd.read_csv(file_path)
    df['geometry'] = df['mgeom'].apply(parse_mgeom)
    df = df.dropna(subset=['geometry'])
    if df.empty:
        print("No valid geometries found.")
        return

    # Initialize boundaries and color mapping for routes
    current_x_max = current_y_max = current_x_min = current_y_min = None
    is_first = True
    num_routes = min(15, len(df))
    colors = plt.cm.jet(np.linspace(0, 1, num_routes))

    # Calculate boundaries based on route data
    for i, geometry in enumerate(df['geometry'].head(num_routes)):
        x, y = geometry.xy
        for lon, lat in zip(x, y):
            current_x_max, current_y_max, current_x_min, current_y_min, is_first = calculate_boundaries(
                current_x_max, current_y_max, current_x_min, current_y_min, is_first, lon, lat
            )

    # Adjust boundaries with padding for the map view
    if is_first:
        # Default boundaries if no points were processed
        current_x_min, current_x_max = -8.7, -8.5  
        current_y_min, current_y_max = 41.1, 41.3
    else:
        x_padding = (current_x_max - current_x_min) * 0.05
        y_padding = (current_y_max - current_y_min) * 0.05
        current_x_min -= x_padding
        current_x_max += x_padding
        current_y_min -= y_padding
        current_y_max += y_padding

    # Generate a color list in hex format for consistency with Folium
    colors_hex = [
        f'#{int(color[0]*255):02x}{int(color[1]*255):02x}{int(color[2]*255):02x}'
        for color in colors
    ]

    # Directory to save individual route maps
    individual_dir = 'data/Task6'
    if not os.path.exists(individual_dir):
        os.makedirs(individual_dir)

    # Create a combined Folium map if required
    if save_combined:
        combined_map = folium.Map(
            location=[(current_y_max + current_y_min) / 2, (current_x_max + current_x_min) / 2],
            zoom_start=13,
            # tiles="cartodbpositron",
            max_bounds=True
        )
        combined_map.fit_bounds([[current_y_min, current_x_min], [current_y_max, current_x_max]])

    # Iterate through each route to add to the combined map and save individually
    for idx, geometry in enumerate(df['geometry'].head(num_routes)):
        lcoord = list(zip(*geometry.xy))  # Convert LineString to list of coordinates (lon, lat)
        lcoord = [(lat, lon) for lon, lat in lcoord]  # Convert to (lat, lon) tuples for Folium
        color = colors_hex[idx]

        # Add to combined map
        if save_combined:
            feature_group = folium.FeatureGroup(name=f'Trip {idx + 1}')

            # Add polyline for the route if plot_lines is enabled
            if plot_lines:
                folium.PolyLine(
                    locations=lcoord,
                    color=color,
                    weight=3,
                    opacity=0.7
                ).add_to(feature_group)

            # Add circle markers for each point if plot_points is enabled
            if plot_points:
                for point in lcoord:
                    folium.CircleMarker(
                        location=point,
                        radius=3,
                        color=color,
                        fill=True,
                        fill_color=color,
                        fill_opacity=1
                    ).add_to(feature_group)

            # Highlight start and end points
            folium.CircleMarker(
                location=lcoord[0],
                radius=5,
                color='green',
                fill=True,
                fill_color='green',
                fill_opacity=1,
                tooltip=f'Start Trip {idx + 1}'
            ).add_to(feature_group)

            folium.CircleMarker(
                location=lcoord[-1],
                radius=5,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=1,
                tooltip=f'End Trip {idx + 1}'
            ).add_to(feature_group)

            feature_group.add_to(combined_map)

        # Save individual map
        if save_individual:
            # Create a Folium map centered on the route's first point
            individual_map = folium.Map(
                location=[lcoord[0][0], lcoord[0][1]],
                zoom_start=14,
                # tiles="cartodbpositron"
            )

            # Adjust map bounds to fit the route
            latitudes = [point[0] for point in lcoord]
            longitudes = [point[1] for point in lcoord]
            bounds = [[min(latitudes), min(longitudes)], [max(latitudes), max(longitudes)]]
            individual_map.fit_bounds(bounds)

            # Create a feature group for the current route
            individual_fg = folium.FeatureGroup(name=f'Trip {idx + 1}')

            # Add polyline for the route if plot_lines is enabled
            if plot_lines:
                folium.PolyLine(
                    locations=lcoord,
                    color=color,
                    weight=3,
                    opacity=0.7
                ).add_to(individual_fg)

            # Add circle markers for each point if plot_points is enabled
            if plot_points:
                for point in lcoord:
                    folium.CircleMarker(
                        location=point,
                        radius=3,
                        color=color,
                        fill=True,
                        fill_color=color,
                        fill_opacity=1
                    ).add_to(individual_fg)

            # Highlight start and end points
            folium.CircleMarker(
                location=lcoord[0],
                radius=5,
                color='green',
                fill=True,
                fill_color='green',
                fill_opacity=1,
                tooltip=f'Start Trip {idx + 1}'
            ).add_to(individual_fg)

            folium.CircleMarker(
                location=lcoord[-1],
                radius=5,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=1,
                tooltip=f'End Trip {idx + 1}'
            ).add_to(individual_fg)

            # Add feature group to the individual map
            individual_fg.add_to(individual_map)

            # Add layer control (optional for individual maps)
            folium.LayerControl().add_to(individual_map)

            # Define output path for individual map
            individual_output_path = os.path.join(individual_dir, f'fmap_route_{idx + 1}.html')
            individual_map.save(individual_output_path)
            print(f"Individual map for Trip {idx + 1} saved to {individual_output_path}")

    # Finalize and save the combined map
    if save_combined:
        # Add layer control to the combined map
        folium.LayerControl().add_to(combined_map)

        # Define output path for combined map
        combined_output_dir = 'data'
        if not os.path.exists(combined_output_dir):
            os.makedirs(combined_output_dir)
        combined_output_path = os.path.join(combined_output_dir, 'Mapped_task6.html')
        combined_map.save(combined_output_path)
        print(f"Combined map saved to {combined_output_path}")

    print("Visualization completed successfully.")

# Run the function with the specified file path
visualize_routes(
    file_path="data/matched_results_1500_cleaned.csv",
    save_combined=True,      
    save_individual=True,    
    plot_lines=True,         
    plot_points=True         
)


Error parsing geometry: IllegalArgumentException: point array must contain 0 or >1 elements

Error parsing geometry: IllegalArgumentException: point array must contain 0 or >1 elements

Individual map for Trip 1 saved to data/Task6\fmap_route_1.html
Individual map for Trip 2 saved to data/Task6\fmap_route_2.html
Individual map for Trip 3 saved to data/Task6\fmap_route_3.html
Individual map for Trip 4 saved to data/Task6\fmap_route_4.html
Individual map for Trip 5 saved to data/Task6\fmap_route_5.html
Individual map for Trip 6 saved to data/Task6\fmap_route_6.html
Individual map for Trip 7 saved to data/Task6\fmap_route_7.html
Individual map for Trip 8 saved to data/Task6\fmap_route_8.html
Individual map for Trip 9 saved to data/Task6\fmap_route_9.html
Individual map for Trip 10 saved to data/Task6\fmap_route_10.html
Individual map for Trip 11 saved to data/Task6\fmap_route_11.html
Individual map for Trip 12 saved to data/Task6\fmap_route_12.html
Individual map for Trip 13 saved to data